<a href="https://colab.research.google.com/github/luck058/kaggle-gold-price-analysis/blob/model-1/model_1_kaggle_gold_price_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


ModuleNotFoundError: No module named 'sklearn'

In [2]:
!git clone https://github.com/luck058/kaggle-gold-price-analysis

%cd kaggle-gold-price-analysis

Cloning into 'kaggle-gold-price-analysis'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 30 (delta 14), reused 13 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (30/30), 667.94 KiB | 9.41 MiB/s, done.
Resolving deltas: 100% (14/14), done.
/content/kaggle-gold-price-analysis


# Create X, y

In [72]:
df = pd.read_csv('financial_regression_cleaned.csv')

# Only care about sp500
df = df[['sp500 open', 'sp500 high', 'sp500 low', 'sp500 close', 'sp500 volume', 'sp500 high-low']].reset_index(drop=True)

display(df.head())
print(df.shape)

,sp500 open,sp500 high,sp500 low,sp500 close,sp500 volume,sp500 high-low
0,114.49,115.14,114.42,114.93,115646960.0,0.72
1,114.73,114.84,113.20,113.64,212252769.0,1.64
2,113.62,115.13,113.59,115.06,138671890.0,1.54
3,114.28,114.45,112.98,113.89,216330645.0,1.47
4,113.92,114.27,111.56,111.70,344747028.0,2.71


(3719, 6)


In [73]:
df["sp500 close diff"] = df["sp500 close"].diff()

display(df.head())
print(df.shape)

,sp500 open,sp500 high,sp500 low,sp500 close,sp500 volume,sp500 high-low,sp500 close diff
0,114.49,115.14,114.42,114.93,115646960.0,0.72,NaN
1,114.73,114.84,113.20,113.64,212252769.0,1.64,-1.29
2,113.62,115.13,113.59,115.06,138671890.0,1.54,1.42
3,114.28,114.45,112.98,113.89,216330645.0,1.47,-1.17
4,113.92,114.27,111.56,111.70,344747028.0,2.71,-2.19


(3719, 7)


In [74]:
def create_lag(df, column, lags):
    # Create Lagged Features
    #creating lag
    for lag in range(lags):
      df[f'{column}-{lag}'] = df[column].shift(lag)

In [75]:
create_lag(df, 'sp500 close diff', 3)
display(df.head())

,sp500 open,sp500 high,sp500 low,sp500 close,sp500 volume,sp500 high-low,sp500 close diff,sp500 close diff-0,sp500 close diff-1,sp500 close diff-2
0,114.49,115.14,114.42,114.93,115646960.0,0.72,NaN,NaN,NaN,NaN
1,114.73,114.84,113.20,113.64,212252769.0,1.64,-1.29,-1.29,NaN,NaN
2,113.62,115.13,113.59,115.06,138671890.0,1.54,1.42,1.42,-1.29,NaN
3,114.28,114.45,112.98,113.89,216330645.0,1.47,-1.17,-1.17,1.42,-1.29
4,113.92,114.27,111.56,111.70,344747028.0,2.71,-2.19,-2.19,-1.17,1.42


## Create y

In [77]:
y_col = "sp500 close diff"

df[y_col] = df[y_col].shift(-1)

display(df.head())

,sp500 open,sp500 high,sp500 low,sp500 close,sp500 volume,sp500 high-low,sp500 close diff,sp500 close diff-0,sp500 close diff-1,sp500 close diff-2
0,114.28,114.45,112.98,113.89,216330645.0,1.47,-2.19,-1.17,1.42,-1.29
1,113.92,114.27,111.56,111.70,344747028.0,2.71,-2.49,-2.19,-1.17,1.42
2,111.20,111.74,109.09,109.21,345627282.0,2.65,0.56,-2.49,-2.19,-1.17
3,110.21,110.41,109.41,109.77,186751367.0,1.00,-0.46,0.56,-2.49,-2.19
4,109.34,110.47,109.04,109.31,210788669.0,1.43,0.52,-0.46,0.56,-2.49


In [82]:
df = df.dropna(axis=0).reset_index(drop=True)
display(df.head())

,sp500 open,sp500 high,sp500 low,sp500 close,sp500 volume,sp500 high-low,sp500 close diff,sp500 close diff-0,sp500 close diff-1,sp500 close diff-2
0,114.28,114.45,112.98,113.89,216330645.0,1.47,-2.19,-1.17,1.42,-1.29
1,113.92,114.27,111.56,111.70,344747028.0,2.71,-2.49,-2.19,-1.17,1.42
2,111.20,111.74,109.09,109.21,345627282.0,2.65,0.56,-2.49,-2.19,-1.17
3,110.21,110.41,109.41,109.77,186751367.0,1.00,-0.46,0.56,-2.49,-2.19
4,109.34,110.47,109.04,109.31,210788669.0,1.43,0.52,-0.46,0.56,-2.49


In [89]:
y = df[y_col]
X = df.drop(y_col, axis=1)

print("y:")
display(y.head())
print("X:")
display(X.head())

print("len(y):")
print(len(y))
print("X.shape:")
print(X.shape)

y:


0   -2.19
1   -2.49
2    0.56
3   -0.46
4    0.52
Name: sp500 close diff, dtype: float64

X:


,sp500 open,sp500 high,sp500 low,sp500 close,sp500 volume,sp500 high-low,sp500 close diff-0,sp500 close diff-1,sp500 close diff-2
0,114.28,114.45,112.98,113.89,216330645.0,1.47,-1.17,1.42,-1.29
1,113.92,114.27,111.56,111.70,344747028.0,2.71,-2.19,-1.17,1.42
2,111.20,111.74,109.09,109.21,345627282.0,2.65,-2.49,-2.19,-1.17
3,110.21,110.41,109.41,109.77,186751367.0,1.00,0.56,-2.49,-2.19
4,109.34,110.47,109.04,109.31,210788669.0,1.43,-0.46,0.56,-2.49


len(y):
3715
X.shape:
(3715, 9)
